# <center> Практическое задание. ML-7. Прогнозирование биологического ответа(HW-03)

#### Примечание:

- Предварительная обработка не требуется, данные уже закодированы и нормализованы.

- В качестве метрики будем использовать F1-score.

## Задание:

Необходимо обучить две модели: логистическую регрессию и случайный лес. 

Далее нужно сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации. 

Важно использовать все четыре метода (GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna) хотя бы по разу, максимальное количество итераций не должно превышать 50.

In [115]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки
from sklearn.linear_model import LogisticRegression #логическая регрессия
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import cross_val_score
from hyperopt import STATUS_OK, hp, fmin, tpe, Trials
import hyperopt
import optuna


%matplotlib inline
plt.style.use('seaborn-v0_8')

## <center> Знакомство с данными
 

In [116]:
data = pd.read_csv("_train_sem09 (1).csv")

In [117]:
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [118]:
# создаем матрицу наблюдений Ч и вектор ответов У
X= data.drop(['Activity'], axis=1)
y= data['Activity']

In [119]:
# разделяем выборку на тестовую и тренировочную в соотношении 80/20.
# Для сохраниения соотношений целевлго признака используем stratify
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 1, test_size = 0.2)

## <center> Логическая регрессия и модель случайного леса

Для начала обучим модель без дополнительных настроек и зафиксируем метрики 


In [120]:
# создаем обьект класса логическая регрессия
log_reg= linear_model.LogisticRegression(max_iter=1000)
# обучаем модель 
log_reg.fit(X_train, y_train)
y_train_pred= log_reg.predict(X_train)
y_test_pred= log_reg.predict(X_test)

# выводим результат метрики f1 score
print('F1-score  на тренировочной выборке : {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('F1-score на тестовой выборке : {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

F1-score  на тренировочной выборке : 0.89
F1-score на тестовой выборке : 0.78


In [121]:
#Создаем объект класса случайный лес 
rf = ensemble.RandomForestClassifier(random_state=42)

#Обучаем модель
rf.fit(X_train, y_train)
#Выводим значения метрики 
y_train_pred = rf.predict(X_train)
print('Train: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = rf.predict(X_test)
print('Test: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Train: 1.00
Test: 0.81


#### **Вывод:**

 ***Логическая регрессия:***

- Значение метрик логической регресии находится в приемлемом диапазоне
- Небольная разница между результатами метрик говорит о том, что переобучения возможно нет. 
- Для возможного увелечения значений метрик мы опробуем разниличные оптимизации гиперпараметров модели 

***Метод случайного леса:***
-На данной модели мы видем явное переобучение. 

### <center> Метод оптимизации GridSearchCV

In [122]:
# создаем гипер параметры 
param_grid = [
              {'penalty': ['l2', 'none'] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регурялизации
              
              {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'],
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
              ]
grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state=42, #генератор случайных чисел
        max_iter=50 #количество итераций на сходимость
    ), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search.fit(X_train, y_train) 

y_train_pred = grid_search.predict(X_train)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))


d:\idi\.venv\lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
70 fits failed out of a total of 280.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
35 fits failed with the following error:
Traceback (most recent call last):
  File "d:\idi\.venv\lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\idi\.venv\lib\site-packages\sklearn\base.py", line 1358, in wrapper
    estimator._validate_params()
  File "d:\idi\.venv\lib\site-packages\sklearn\base.py", line 471, in _validate_params
    validate_parameter_constraints(
  File "d:\idi\.venv\lib\site-packages\sklearn\utils\_param_validation.py", line 98, in validate_p

CPU times: total: 2.09 s
Wall time: 33.9 s
f1_score на тестовом наборе: 0.85
f1_score на тестовом наборе: 0.79


d:\idi\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 50 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=50).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [123]:
# создадим сетку гипер параметров 
param_distributions  = { 'n_estimators': list(range(100,200,30)),
                        'min_samples_leaf' : [5,7] , 
                        'max_depth' : [10,15,20,25]
                        }

#  посчитаем метрику 
grid_search_forest= GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42),
    param_grid=param_distributions, 
    cv=5,
    n_jobs=-1
)
 
%time grid_search_forest.fit(X_train,y_train)


y_train_pred= grid_search_forest.predict(X_train)
print('F1-scores на тренир. выборке для случайного леса в GrigSearchCV: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred= grid_search_forest.predict(X_test)
print('F1-scores на тестовой выборке для случайного леса в GrigSearchCV: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

CPU times: total: 1.61 s
Wall time: 32 s
F1-scores на тренир. выборке для случайного леса в GrigSearchCV: 0.93
F1-scores на тестовой выборке для случайного леса в GrigSearchCV: 0.82


#### **Вывод:**

***Логическая регрессия:***
- Время работы 32.5 s
- f1_score на тренир наборе: 0.85
- f1_score на тестовом наборе: 0.79
- Значение целевой метрики на тестовой выборке увеличилось на 0.01
- значение целевой метрики на тренировочной выборке уменьшилось на 0,04, модель лучше начала находить зависимости в данных.
- Лучшими значениями гипер параметров являются: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}


***Метод случайного леса:***
- Время работы 32.7 s
- F1-scores на тренир. выборке для случайного леса в GrigSearchCV: 0.93  
- F1-scores на тестовой выборке для случайного леса в GrigSearchCV: 0.82
- Значение целевой метрики на тренировочной выборке значительно уменьшилось,  что может говорить о том,  что  модель лучше начала находить зависимости в данных. 
- Значение целевой метрики на тестовой выборке увеличилось на 0.01. 


         Метод случайно леса лучше находит зависимости в тестовой выборке но при этом хуже работает с тренировочными данными 


### <center> Метод оптимизации **RandomizedSearchCV**

***Модель логической регрессии***

In [124]:

param_distributions =  [
              {'penalty': ['l2', 'none'] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регурялизации
              
              {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'],
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
              ]
            
random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=50), 
    param_distributions=param_distributions, 
    cv=5, 
    n_iter = 50, 
    n_jobs = -1
)  
%time random_search.fit(X_train, y_train) 
y_train_pred = random_search.predict(X_train)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

d:\idi\.venv\lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
65 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
19 fits failed with the following error:
Traceback (most recent call last):
  File "d:\idi\.venv\lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "d:\idi\.venv\lib\site-packages\sklearn\base.py", line 1358, in wrapper
    estimator._validate_params()
  File "d:\idi\.venv\lib\site-packages\sklearn\base.py", line 471, in _validate_params
    validate_parameter_constraints(
  File "d:\idi\.venv\lib\site-packages\sklearn\utils\_param_validation.py", line 98, in validate_p

CPU times: total: 922 ms
Wall time: 30.5 s
f1_score на тестовом наборе: 0.85
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'solver': 'liblinear', 'penalty': 'l2', 'C': 0.1}


***Метод случайного леса***


In [125]:
param_distributions = {'n_estimators': list(range(100, 200, 30)),
              'min_samples_leaf': [5, 7],
              'max_depth': [10,15,20,25]
              }
            
random_search_forest = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_distributions, 
    cv=5,
    n_jobs = -1,
    n_iter=50
)  
%time random_search_forest.fit(X_train, y_train) 
y_train_pred = random_search_forest.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = random_search_forest.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_forest.best_params_))

d:\idi\.venv\lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 32 is smaller than n_iter=50. Running 32 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


CPU times: total: 1.7 s
Wall time: 31.7 s
f1_score на обучающем наборе: 0.93
f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'n_estimators': 130, 'min_samples_leaf': 7, 'max_depth': 20}


Метод оптимизации **RandomizedSearchCV**

***Логическая регрессия:***
- Время работы 26.8 s
- f1_score на тестовом наборе: 0.85
- f1_score на тестовом наборе: 0.79
- Значение целевой метрики осталось без изменений но модель стала работать быстрее 
- Наилучшие значения гиперпараметров: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.1}


***Метод случайного леса:***
- Время работы 31.8 s
- f1_score на обучающем наборе: 0.93
- f1_score на тестовом наборе: 0.82
- Наилучшие значения гиперпараметров: {'n_estimators': 130, 'min_samples_leaf': 7, 'max_depth': 20}
- Значение целевой метрики осталось без изменений но модель стала работать быстрее 


        значение метрик осталось без  изменений, но для работы модель стала требовать меньше времени 

### <center> Метод оптимизации Hyperopt

***Модель логической регрессии***

In [126]:

#  пространство параметров для Hyperopt
space_lr = {
    'penalty': hp.choice('penalty', ['l1', 'l2', 'none']),
    'solver': hp.choice('solver', ['lbfgs', 'liblinear', 'sag', 'saga']),
    'C': hp.uniform('C', 0.01, 1.0),
}

# Зафиксируем random_state
random_state = 42

def hyperopt_lr(params, X=X_train, y=y_train, cv=5):
    """
    Функция для оптимизации гиперпараметров логистической регрессии
    """
    # Базовые параметры
    model_params = {
        'C': float(params.get('C', 1.0)),
        'penalty': params.get('penalty', 'l2'),
        'solver': params.get('solver', 'lbfgs'),
        'max_iter': 50, 
        'random_state': random_state,
        'n_jobs': -1  # Используем все ядра
    }
    
    # Проверка совместимости penalty и solver
    penalty = model_params['penalty']
    solver = model_params['solver']
    
    # Обработка несовместимых комбинаций
    if penalty == 'l1' and solver not in ['liblinear', 'saga']:
        model_params['solver'] = 'liblinear'
    elif penalty == 'none' and solver not in ['lbfgs', 'sag', 'saga']:
        model_params['solver'] = 'lbfgs'
    elif penalty == 'elasticnet':
        # elasticnet требует solver='saga'
        model_params['solver'] = 'saga'
    
    # Обработка 'none' penalty (требует solver, который поддерживает no penalty)
    if penalty == 'none':
        model_params['penalty'] = None
        # Для sklearn >= 1.0 необходимо установить penalty=None
        if 'l1_ratio' in model_params:
            del model_params['l1_ratio']
    
    #  строим модель логистической регрессии 
    model = LogisticRegression(**model_params)
    
    # Используем кросс-валидацию вместо обучения на всех данных
    try:
        scores = cross_val_score(model, X, y, 
                               cv=cv, 
                               scoring='f1',
                               n_jobs=-1)
        mean_score = np.mean(scores)
        return {'loss': -mean_score, 'status': STATUS_OK}
    except Exception as e:
        # В случае ошибки возвращаем плохой результат
        return {'loss': 0, 'status': STATUS_OK}

# Инициализация
trials = Trials()

# Запуск оптимизации
best = fmin(
    fn=hyperopt_lr,  # наша функция
    space=space_lr,  # пространство параметров
    algo=tpe.suggest,  # алгоритм оптимизации
    max_evals=50,  # количество итераций
    trials=trials,
    rstate=np.random.default_rng(random_state)
)

print("Лучшие параметры:", best)

# Чтобы получить читаемые имена параметров из choice
best_params_readable = {
    'C': best['C'],
    'penalty': ['l1', 'l2', 'none'][best['penalty']],
    'solver': ['lbfgs', 'liblinear', 'sag', 'saga'][best['solver']]
}
print("Лучшие параметры (читаемые):", best_params_readable)


100%|██████████| 50/50 [01:03<00:00,  1.27s/trial, best loss: -0.7828053776216576]
Лучшие параметры: {'C': 0.21224354014919164, 'penalty': 0, 'solver': 0}
Лучшие параметры (читаемые): {'C': 0.21224354014919164, 'penalty': 'l1', 'solver': 'lbfgs'}


In [127]:
# рассчитаем точность на тестовой выборке 
model = linear_model.LogisticRegression(
    C=0.212, penalty='l2', solver='lbfgs' , max_iter= 50, random_state=42  )
model.fit(X_train, y_train)
y_train_pred= model.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred_2= model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred_2)))

f1_score на обучающем наборе: 0.86
f1_score на тестовом наборе: 0.79


d:\idi\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 50 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=50).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [128]:
# рассчитаем точность на тестовой выборке 
model_2 = linear_model.LogisticRegression(
    C=0.212, penalty='l1', solver='liblinear' , max_iter= 50, random_state=42  )
model_2.fit(X_train, y_train)
y_train_pred= model_2.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred= model_2.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на обучающем наборе: 0.82
f1_score на тестовом наборе: 0.79


***Метод  случайного леса***

In [129]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 10, 30, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 15, 1)
      }

In [130]:
# зафксируем random_state
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    score = metrics.f1_score(y, model.predict(X))

    return -score

In [132]:
%%time
# начинаем подбор гиперпараметров

trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals= 50, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
        )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 50/50 [01:19<00:00,  1.60s/trial, best loss: -0.9929555895865237]
Наилучшие значения гиперпараметров {'max_depth': 25.0, 'min_samples_leaf': 2.0, 'n_estimators': 195.0}
CPU times: total: 1min 19s
Wall time: 1min 19s


In [133]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на обучающем наборе: 0.99
f1_score на тестовом наборе: 0.83


Метод оптимизации **Hyperopt**

***Логическая регрессия:***
- Лучшие параметры : { C=0.212, penalty='l1', solver='liblinear' }
- f1_score на обучающем наборе: 0.82
- f1_score на тестовом наборе: 0.79
- Время работы 67 s
- Модель работает дольше, значение метрики изменилось только для обучающей выборки( на 0,07 от первоначальной модели). Разница в значениях говорит,  о том ,  что модель хорошо обучается и в ней нет переобучения


***Метод случайного леса***
- Наилучшие значения гиперпараметров {'max_depth': 25.0, 'min_samples_leaf': 2.0, 'n_estimators': 195.0}
- Время работы 29.4 s
- f1_score на обучающем наборе: 0.99
- f1_score на тестовом наборе: 0.83
- значения метрики на тестовой выборке изменилось на 0,01 ,  но при этом метрика обучающей выборке увеличилась по сравнению с прошлыми моделями это говорит, что модель переобучена


### <center> Optuna

***Логическая регрессия:***


In [143]:
def optuna_lr(trial):
    """
    Подбирает лучшие параметры для логистической регрессии с помощью Optuna
    """
    # Подбираем гиперпараметры через trial
    model_params = {
        'C': trial.suggest_float('C', 0.01, 10.0), 
        'penalty': trial.suggest_categorical('penalty', ['l2', 'l1', 'elasticnet', 'none']),
        'solver': trial.suggest_categorical('solver', ['lbfgs', 'liblinear', 'saga']),
        'max_iter': 50, 
        'random_state': random_state,
        'n_jobs': -1  # Работаем на всех ядрах процессора
    }
    
    # Если выбран elasticnet, нужно добавить l1_ratio
    if model_params['penalty'] == 'elasticnet':
        model_params['l1_ratio'] = trial.suggest_float('l1_ratio', 0.0, 1.0)
    
    penalty = model_params['penalty']
    solver = model_params['solver']
    
    # Исправляем несовместимые сочетания penalty и solver
    if penalty == 'l1' and solver not in ['liblinear', 'saga']:
        model_params['solver'] = 'liblinear'
        
    elif penalty == 'none' and solver not in ['lbfgs', 'sag', 'saga']:
        model_params['solver'] = 'lbfgs'
        
    elif penalty == 'elasticnet':
        model_params['solver'] = 'saga'
    
    # Особый случай: когда penalty='none'
    if penalty == 'none':
        model_params['penalty'] = None
        # Удаляем l1_ratio, если он есть (для sklearn версии 1.0 и выше)
        if 'l1_ratio' in model_params:
            del model_params['l1_ratio']
    
    # Создаём и обучаем модель
    model = LogisticRegression(**model_params)
    model.fit(X_train, y_train)
    score = metrics.f1_score(y_train, model.predict(X_train))

    return score
        


In [144]:
%%time



# Создаем объект исследования
study = optuna.create_study(
    study_name="LogisticRegression",
    direction="maximize"  # Хотим максимизировать F1-score
)

# Ищем лучшую комбинацию гиперпараметров
study.optimize(optuna_lr, n_trials=30)



[I 2026-01-23 13:43:20,548] A new study created in memory with name: LogisticRegression
d:\idi\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:1232: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
[I 2026-01-23 13:43:20,963] Trial 0 finished with value: 0.8775698066891685 and parameters: {'C': 2.9173846215379124, 'penalty': 'none', 'solver': 'liblinear'}. Best is trial 0 with value: 0.8775698066891685.
d:\idi\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:1305: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  warnings.warn(
[I 2026-01-23 13:43:27,333] Trial 1 finished with value: 0.9411042944785276 and parameters: {'C': 9.831437920111984, 'penalty': 'l1', 'solver': 'lbfgs'}. Best is trial 1 with value: 0.9411042944785276.
d:\idi\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:1232: UserWarning: Setting penalty=None will ignore the C and l1_ratio parame

CPU times: total: 1min 9s
Wall time: 1min 12s


In [145]:
# Показываем лучший результат
print(f"Лучшее значение F1: {study.best_value:.2f}")
print("Лучшие параметры:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

Лучшее значение F1: 0.94
Лучшие параметры:
  C: 9.995305615959083
  penalty: l1
  solver: lbfgs


***Метод случайного леса***

In [ ]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 10, 30, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train))

  return score
  

In [ ]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 300, 10)
  max_depth = trial.suggest_int('max_depth', 15, 40, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 3, 7, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train))

  return score

In [ ]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))


[I 2026-01-23 13:03:14,253] A new study created in memory with name: RandomForestClassifier
C:\Users\Admin\AppData\Local\Temp\ipykernel_4652\2278173580.py:3: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
Positional arguments ['self', 'name', 'low', 'high', 'step', 'log'] in suggest_int() have been deprecated since v3.5.0. They will be replaced with the corresponding keyword arguments in v5.0.0, so please use the keyword specification instead. See https://github.com/optuna/optuna/releases/tag/v3.5.0 for details.
  n_estimators = trial.suggest_int('n_estimators', 100, 300, 10)
C:\Users\Admin\AppData\Local\Temp\ipykernel_4652\2278173580.py:4: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
Positional arguments ['self', 'name', 'low', 'high', 'step', 'log'] in suggest_int() have been deprecated since v3.5.0. They will be replaced with the c

Наилучшие значения гиперпараметров {'n_estimators': 280, 'max_depth': 30, 'min_samples_leaf': 3}
CPU times: total: 51.6 s
Wall time: 52.1 s


In [ ]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на обучающем наборе: 0.98
f1_score на тестовом наборе: 0.82



Метод оптимизации **Optuna**

***Логическая регрессия:***
- Лучшее значение F1: 0.79
- Лучшие параметры:{'C': 3.875626519603782, 'penalty': 'none', 'solver': 'lbfgs'}.
- Время работы
-- значения метрики на тестовой выборке не изменилось по сравнению с др моделями, но значения метрики достаточно высоко и его можно оставить если оно удовлетворяет нашим условиям.

- ***Метод случайного леса***
- - Время работы : 52.1 s
- f1_score на обучающем наборе: 0.98
-  f1_score на тестовом наборе: 0.82
- Наилучшие значения гиперпараметров {'n_estimators': 280, 'max_depth': 30, 'min_samples_leaf': 3}
-- значения метрики на тестовой выборке не изменилось по сравнению с др моделями ,  но при этом метрика обучающей выборке увеличилась по сравнению с прошлыми моделями это говорит, что модель переобучена



### ***ОБЩИЙ ВЫВОД***

**Для логистической регрессии:**

Лучший метод: **RandomizedSearchCV**

Причина: F1-score на тестовой выборке (0.79) такой же, как и у других методов, но время работы (26.8 с) значительно меньше

Дополнительные преимущества:

- Не уступает по качеству GridSearchCV и Hyperopt
- В 2.5 раза быстрее Hyperopt (67 с) и быстрее GridSearchCV (32.5 с)
- Производительность аналогична Optuna, но с лучшей воспроизводимостью

**Для случайного леса:**

Лучший метод: **Hyperopt**

Причина: Самый высокий F1-score на тестовой выборке (0.83) среди всех методов

Время работы приемлемое: 29.4 с (второе место по скорости после RandomizedSearchCV)

Примечание: Хотя модель показывает признаки переобучения (0.99 на обучающей выборке), она дает наилучший результат на тестовых данных

Общий вывод:
RandomizedSearchCV лучше всего подходит для логистической регрессии — оптимальное соотношение скорости и качества

Hyperopt лучше всего подходит для случайного леса — обеспечивает наивысшее качество предсказаний

Optuna показала хорошие результаты для логистической регрессии, но не имеет значительных преимуществ перед RandomizedSearchCV

GridSearchCV требует больше времени без значительного улучшения качества

Рекомендация: Использовать RandomizedSearchCV для логистической регрессии и Hyperopt для случайного леса, так как это дает наилучшее сочетание производительности и качества моделей.